In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from PIL import Image, ImageFile
import requests
from io import BytesIO

# Handle truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Paths for datasets
train_dataset_path = r"biodnonbiodddatset/train"
test_dataset_path = r"biodnonbiodddatset/val"

# Waste categories (must match dataset folder names)
waste_categories = ['biodegradable', 'non_biodegradable']
num_classes = len(waste_categories)

# Image dimensions
img_width, img_height = 128, 128
input_shape = (img_width, img_height, 3)

# Build CNN Model
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Compile the model
model = build_cnn_model(input_shape, num_classes)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

# Debug: Check Detected Classes
print("Class labels detected by train generator:", train_generator.class_indices)
print("Class labels detected by test generator:", test_generator.class_indices)

# Train the Model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Save the Model
model.save('waste_classification_model.keras')

# Load the Model for Testing
model = tf.keras.models.load_model('waste_classification_model.keras')

# Function to Preprocess Images for Classification
def validate_and_preprocess_image(image_path_or_url):
    try:
        if image_path_or_url.startswith(('http://', 'https://')):
            response = requests.get(image_path_or_url)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
        else:
            img = Image.open(image_path_or_url)

        img = img.convert('RGB')
        img = np.array(img)

        # Resize and normalize the image
        img = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_LINEAR)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        return img
    except Exception as e:
        print(f"Invalid image: {e}")
        return None

# Function to Classify Waste Type
def classify_waste(image_path_or_url):
    img = validate_and_preprocess_image(image_path_or_url)
    if img is None:
        return {"status": "error", "message": "Invalid image. Please upload a valid image."}

    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])
    waste_type = waste_categories[predicted_class]
    confidence = float(predictions[0][predicted_class])

    return {"status": "success", "waste_type": waste_type, "confidence": confidence}

# Example Usage
image_path = r"biodnonbiodddatset\val\biodegradable\paper_waste\paper567.jpg"
result = classify_waste(image_path)

# Display Output
if result["status"] == "success":
    print(f"Detected Waste Type: {result['waste_type']} (Confidence: {result['confidence']:.2%})")
else:
    print("Error:", result["message"])


In [11]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2
import requests
from io import BytesIO

# Image dimensions
img_width, img_height = 128, 128
input_shape = (img_width, img_height, 3)

# Waste categories (must match folder names used during training)
waste_categories = ['biodegradable', 'non_biodegradable']

# Load the saved model
model = tf.keras.models.load_model('waste_classification_model.keras')

# Function to preprocess the image
def validate_and_preprocess_image(image_path_or_url):
    try:
        if image_path_or_url.startswith(('http://', 'https://')):
            response = requests.get(image_path_or_url)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))
        else:
            if not os.path.exists(image_path_or_url):
                print(f"Error: File not found - {image_path_or_url}")
                return None
            img = Image.open(image_path_or_url)

        img = img.convert('RGB')
        img = np.array(img)

        # Resize and normalize
        img = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_LINEAR)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        return img
    except Exception as e:
        print(f"Invalid image: {e}")
        return None

# Function to classify waste
def classify_waste(image_path_or_url):
    img = validate_and_preprocess_image(image_path_or_url)
    if img is None:
        return {"status": "error", "message": "Invalid or missing image."}

    # Make predictions on the new image
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])  # Get index of class with highest probability
    waste_type = waste_categories[predicted_class]
    confidence = float(predictions[0][predicted_class])

    return {"status": "success", "waste_type": waste_type, "confidence": confidence}

# Example usage
image_path = r"biodnonbiodddatset\train\non_biodegradable\ewaste\00000255.jpg"  # Replace with the path to your image

if os.path.exists(image_path):
    result = classify_waste(image_path)
    print(f"Detected Waste Type: {result['waste_type']} (Confidence: {result['confidence']:.2%})")
else:
    print("Error: Image file does not exist. Please check the path.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step
Detected Waste Type: non_biodegradable (Confidence: 83.98%)
